https://colab.research.google.com/github/sberbank-ai/ru-gpts/blob/master/examples/Finetune_RuGPTs_with_HF.ipynb#scrollTo=aZ-4Kav28cH0




# **IMPORT**

In [1]:
!apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease                
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease              
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done                     


In [2]:
!apt-get -y install wget

Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.19.4-1ubuntu2.2).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [3]:
!apt-get -y install git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.9).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [4]:
!pip install pandas

In [5]:
!pip install sklearn

In [6]:
!pip install pandas

In [7]:
import pandas as pd

In [8]:
!pip install transformers -U

In [9]:
!pip install sentencepiece

In [10]:
!pip install tqdm boto3 requests regex sacremoses

In [11]:
import torch
from transformers import AutoModel, AutoTokenizer, AutoConfig

In [12]:
# model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 'gpt2')   
#  # Download model and configuration from huggingface.co and cache.
# # model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', './test/saved_model/')  
# # E.g. model was saved using `save_pretrained('./test/saved_model/')`
# model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 'gpt2', output_attentions=True)  
# # Update configuration during loading
# assert model.config.output_attentions == True
# # Loading from a TF checkpoint file instead of a PyTorch model (slower)
# config = AutoConfig.from_pretrained('bert-base-uncased')
# # AutoConfig.from_pretrained('bert-base-uncased')
# # model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 
# #                        './tf_model/gpt_tf_checkpoint.ckpt.index', from_tf=True, config=config)

In [13]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 93484, done.
remote: Total 93484 (delta 0), reused 0 (delta 0), pack-reused 93484
Receiving objects: 100% (93484/93484), 78.66 MiB | 32.89 MiB/s, done.
Resolving deltas: 100% (67661/67661), done.


In [15]:
!pip install natasha

In [16]:
from natasha import (   
    MorphVocab,
    NewsNERTagger,
    NamesExtractor,
    MorphVocab,
    Segmenter, 
    NewsEmbedding,
    PER,
    Doc,
    ORG
)

In [17]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# **PREPARE DATA SET**

In [18]:
!wget https://raw.githubusercontent.com/DmitryKutsev/congrats_generator/main/crawler/curr_congrats.txt

--2021-12-25 10:04:25--  https://raw.githubusercontent.com/DmitryKutsev/congrats_generator/main/crawler/curr_congrats.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1471875 (1.4M) [text/plain]
Saving to: ‘curr_congrats.txt’

curr_congrats.txt   100%[===================>]   1.40M  --.-KB/s    in 0.1s    

2021-12-25 10:04:26 (9.49 MB/s) - ‘curr_congrats.txt’ saved [1471875/1471875]



In [19]:
with open('curr_congrats.txt') as handler:
  full_text = handler.read()
full_text[:3000]

'\nTITLE\nЭдварду Радзинскому, писателю, драматургу, сценаристу\nCONTENT\n Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанных\nдраматургов, мастеров современного театрального искусства. И\xa0конечно, Вас\nхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьё\nтворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодрости\nдуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      \nEND\n\nTITLE\nКоллективу радиостанции «Орфей»\nCONTENT\n Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «Орфей».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вклад в\xa0сбережение луч

In [20]:
full_text.split('END')[:3][0].replace('\n', '').split('CONTENT')

['TITLEЭдварду Радзинскому, писателю, драматургу, сценаристу',
 ' Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, Васхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ']

In [21]:
titles_list = []
congrats_list = []

for letter in full_text.split('END'):
  letter = letter.replace('\n', '').replace('TITLE', '')

  if len(letter.split('CONTENT')) > 1:
    titles_list.append(letter.split('CONTENT')[0])
    congrats_list.append(letter.split('CONTENT')[1])

print(titles_list[:3])
print(congrats_list[:3])
print(len(titles_list))
print(len(congrats_list))

['Эдварду Радзинскому, писателю, драматургу, сценаристу', 'Коллективу радиостанции «Орфей»', 'Михаилу Ковальчуку, президенту НИЦ «Курчатовский институт»']
[' Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, Васхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ', ' Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «Орфей».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вкла

In [22]:
data = {'Title': titles_list,
        'Content': congrats_list}

my_df = pd.DataFrame(data)
my_df

,Title,Content
0,"Эдварду Радзинскому, писателю, драматургу, сце...",Уважаемый Эдвард Станиславович!Примите поздра...
1,Коллективу радиостанции «Орфей»,Дорогие друзья!Поздравляю вас со знаменательн...
2,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск...",Уважаемый Михаил Валентинович!Примите мои поз...
3,"Ирине Андреевой и Ивану Штылю, победителям чем...",Уважаемые Ирина Александровна и Иван Александ...
4,Работникам и ветеранам оборонно-промышленного ...,Уважаемые друзья!Поздравляю вас с Днём оружей...
...,...,...
883,Жителям Республики Мордовия,Дорогие друзья!Поздравляю вас с юбилеем – 75-...
884,Коллективу Института физических проблем им.П.Л...,Поздравляю коллектив института с 70-летием со...
885,Жителям Красноярского края,Уважаемые друзья!Поздравляю вас с 70-летием с...
886,В.А.ЛОГИНОВУ,Уважаемый Владимир Александрович!Поздравляю В...


In [23]:
!pip install datasets transformers

In [24]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 683, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 683 (delta 110), reused 141 (delta 83), pack-reused 505
Receiving objects: 100% (683/683), 413.81 KiB | 2.11 MiB/s, done.
Resolving deltas: 100% (410/410), done.


In [25]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import re


In [26]:
my_df['Sum'] = [my_df['Title'].tolist()[i] + my_df['Content'].tolist()[i] for i in range(len(my_df))]
my_df

,Title,Content,Sum
0,"Эдварду Радзинскому, писателю, драматургу, сце...",Уважаемый Эдвард Станиславович!Примите поздра...,"Эдварду Радзинскому, писателю, драматургу, сце..."
1,Коллективу радиостанции «Орфей»,Дорогие друзья!Поздравляю вас со знаменательн...,Коллективу радиостанции «Орфей» Дорогие друзья...
2,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск...",Уважаемый Михаил Валентинович!Примите мои поз...,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск..."
3,"Ирине Андреевой и Ивану Штылю, победителям чем...",Уважаемые Ирина Александровна и Иван Александ...,"Ирине Андреевой и Ивану Штылю, победителям чем..."
4,Работникам и ветеранам оборонно-промышленного ...,Уважаемые друзья!Поздравляю вас с Днём оружей...,Работникам и ветеранам оборонно-промышленного ...
...,...,...,...
883,Жителям Республики Мордовия,Дорогие друзья!Поздравляю вас с юбилеем – 75-...,Жителям Республики Мордовия Дорогие друзья!Поз...
884,Коллективу Института физических проблем им.П.Л...,Поздравляю коллектив института с 70-летием со...,Коллективу Института физических проблем им.П.Л...
885,Жителям Красноярского края,Уважаемые друзья!Поздравляю вас с 70-летием с...,Жителям Красноярского края Уважаемые друзья!По...
886,В.А.ЛОГИНОВУ,Уважаемый Владимир Александрович!Поздравляю В...,В.А.ЛОГИНОВУ Уважаемый Владимир Александрович!...


# **NER**

In [27]:
emb = NewsEmbedding()
morph_vocab = MorphVocab()

names_extractor = NamesExtractor(morph_vocab)
ner_tagger = NewsNERTagger(emb)

In [28]:
text = "Эдварду Радзинскому, писателю, драматургу, сценаристуКоллективу радиостанции «Орфей», Михаилу Ковальчуку, президенту НИЦ «Курчатовский институт»"
doc = Doc(text)

In [29]:
doc

Doc(text='Эдварду Радзинскому, писателю, драматургу, сценар...)

In [30]:
segmenter = Segmenter()
doc.segment(segmenter)

In [31]:
doc.tag_ner(ner_tagger)

In [32]:
for span in doc.spans:
    print(span)
    # if span.type == PER:
    #   print(span)


DocSpan(stop=19, type='PER', text='Эдварду Радзинскому', tokens=[...])
DocSpan(start=78, stop=83, type='ORG', text='Орфей', tokens=[...])
DocSpan(start=86, stop=104, type='PER', text='Михаилу Ковальчуку', tokens=[...])
DocSpan(start=117, stop=144, type='ORG', text='НИЦ «Курчатовский институт»', tokens=[...])


***Можно маскировать как имена, так и оргинизации. Попробую для начала и то, и другое вместе.***

In [33]:
# masked_content = []

# for sent in my_df['Content']:
#   doc = Doc(sent)
#   doc.segment(segmenter)
#   doc.tag_ner(ner_tagger)

#   for span in doc.spans:
#       if span.type == PER or span.type == ORG:
#           sent = sent.replace(span.text, 'MASK')

#   masked_content.append(sent)


***Title оставляем прежним, маскируем контент. Модели будет скармливаться колонка с конкатенированным текстом.***

In [34]:
# my_df['Masked_content'] = masked_content
# my_df['Sum'] = [my_df['Title'].tolist()[i] + my_df['Masked_content'].tolist()[i] for i in range(len(my_df))]
my_df['Sum'] = [my_df['Title'].tolist()[i] + my_df['Content'].tolist()[i] for i in range(len(my_df))]
my_df['Sum'].to_list()[:3]

['Эдварду Радзинскому, писателю, драматургу, сценаристу Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, Васхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ',
 'Коллективу радиостанции «Орфей» Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «Орфей».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вклад в\xa0сбережение лучших традиций отечественной культуры. Вы сформиров

# **PREPARE DATA FOR TRAINING**

In [35]:
train_test_ratio = 0.9
df_train, df_test = train_test_split(my_df, train_size = train_test_ratio, random_state = 1)
# train_valid_ratio = 7/9
# df_train, df_valid = train_test_split(df_full_train, train_size = train_valid_ratio, random_state = 1)

***Первичная очистка данных от мусора типа "Ссылка на метериал"***

In [36]:
def build_dataset(df, dest_path):
    f = open(dest_path, 'w')
    data = ''
    summaries = df['Sum'].tolist()

    for summary in summaries:
        summary = str(summary).strip()
        summary = re.sub(r"Статус материала Опубликован в разделе: Телеграммы", " ", summary)
        summary = re.sub(r"Ссылка на материал:", " ", summary)
        summary = re.sub(r"Текстовая версия:", " ", summary)
        summary = re.sub(r"Текстовая версия", " ", summary)
        summary = re.sub(r"Ссылка на материал:", " ", summary)
        summary = re.sub(r"\xa0—", " ", summary)
        summary = re.sub(r"\s", " ", summary)
        summary = re.sub(r"\n", " ", summary)
        summary = re.sub(r"\n+", " ", summary)
        # summary = re.sub(r"MASK", "МАСК", summary)
        summary = re.sub(r"([a-zA-Z]+)", " ", summary)
        # summary = re.sub(r"МАСК", "MASK", summary)
        summary = re.sub(r"([/*#@+]+)", " ", summary)
        summary = re.sub(r" +", " ", summary)
        data += summary

    f.write(data)

In [37]:
!ls
!rm 'train.txt' 'test.txt'
!ls

 curr_congrats.txt   onstart.log   ru-gpts    train.txt
'my_gpt (1).ipynb'   onstart.sh    test.txt   transformers
 curr_congrats.txt   onstart.log   ru-gpts
'my_gpt (1).ipynb'   onstart.sh    transformers


In [38]:
build_dataset(df_train, 'train.txt')
build_dataset(df_test, 'test.txt')
# build_dataset(df_valid, 'valid.txt')

***Проверка тескста на наличие MASK***

In [39]:
with open('train.txt') as handler:
  readed = handler.read()
readed[:1037]


'Коллективу «Российской газеты» Дорогие друзья!Поздравляю вас со знаменательной датой – 25-летиемсоздания «Российской газеты».За прошедшие годы газета стала одним из ведущихпериодических изданий страны, завоевала уважение большой читательской аудиториии авторитет в отечественном медиасообществе. Её ценят за информационнуюнаполненность, глубокий подход к освещению важнейших тем и событий, за взвешенность и основательность оценок. Отрадно, что все эти годы ваш коллективвысоко держит планку качества работы, сохраняет приверженность лучшим традициямжурналистики, расширяет линейку информационных продуктов – от публикациигосударственных документов до новостей, репортажей и серьёзной аналитики.Уверен, что вы и впредь будете добросовестнотрудиться, воплощать в жизнь востребованные творческие проекты, добиватьсяуспеха в профессиональной деятельности.Желаю вам всего наилучшего. Владимир Путин Темы СМИ Статус материала Опубликован в разделе: Телеграммы . 50661 Работникам и ветеранам автомобильног

In [40]:
# tokenizer = AutoTokenizer.from_pretrained("anonymous-german-nlp/german-gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
 
train_path = 'train.txt'
test_path = 'test.txt'

In [41]:
!rm ./gpt-letters

rm: cannot remove './gpt-letters': No such file or directory


In [42]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/opt/conda/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


## **TRAIN MODEL**

In [43]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")


training_args = TrainingArguments(
    output_dir="./gpt-letters", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    # num_train_epochs=3, # number of training epochs
    # per_device_train_batch_size=32, # batch size for training
    # per_device_eval_batch_size=64,  # batch size for evaluation
    num_train_epochs=15, # number of training epochs
    per_device_train_batch_size=64, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=150, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272168290/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [44]:
trainer.train()

***** Running training *****
  Num examples = 1039
  Num Epochs = 15
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 255


Step,Training Loss


Saving model checkpoint to ./gpt-letters/checkpoint-150
Configuration saved in ./gpt-letters/checkpoint-150/config.json
Model weights saved in ./gpt-letters/checkpoint-150/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=255, training_loss=2.2716456992953433, metrics={'train_runtime': 10215.0542, 'train_samples_per_second': 1.526, 'train_steps_per_second': 0.025, 'total_flos': 1018059079680000.0, 'train_loss': 2.2716456992953433, 'epoch': 15.0})

In [45]:
trainer.save_model()

Saving model checkpoint to ./gpt-letters
Configuration saved in ./gpt-letters/config.json
Model weights saved in ./gpt-letters/pytorch_model.bin


In [46]:
# my_model = torch.load('./gpt-letters/pytorch_model.bin')
from transformers import pipeline

pu = pipeline('text-generation',model='./gpt-letters', tokenizer=tokenizer, max_length=500)

loading configuration file ./gpt-letters/config.json
Model config GPT2Config {
  "_name_or_path": "./gpt-letters",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "use_cache": true,
  "vocab_size": 50264
}

loading configuration file ./gpt-letters/config.json
Model config GPT2Config {
  "_

In [47]:
pu("Александру Сергеевичу Пушкину")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Александру Сергеевичу Пушкину, народному художнику России Уважаемый Александр Сергеевич!Примите поздравления с 75-летием.Вы посвятили свою жизньискусству, создали замечательные полотна, которые по достоинству оценили и ценители прекрасного.Вас знают и любят как талантливого художника, и как замечательного педагога, который бережнохранит и приумножает богатейшее наследие своих предшественников.Желаю Вам здоровья, благополучия и всего самого доброго. Владимир Путин Статус материала Опубликован в разделе: Телеграммы. 66521 Александру Лукашенко, Президенту Республики Беларусь Уважаемый Александр Григорьевич!Примите поздравления с 75-летием.Вы по праву принадлежите к плеяде выдающихсялитературоведов, историков, филологов,искусствоведов,искусствоведов,искусствоведов,искусствоведов,искусствоведов,искусствоведов,специалистов в области истории и культуры Беларуси.Вас знают и любят как талантливого писателя,искусствоведа,искусствоведа,специалиста в области истории и культуры Беларуси.Вас ценят 

In [48]:
pu("Кровавым убийцам детей")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Кровавым убийцам детей, которые в течение десятилетий своими злодеяниями снискали заслуженное уважение и любовь миллионов людей.Желаю вам новых свершений и всего самого доброго. Владимир Путин Статус материала Опубликован в разделе: Телеграммы. 66666 Коллективу и ветеранам АО «АВИТЕК» Дорогие друзья!Поздравляю вас с 20-летием АО «АВИТЕК».За прошедшие годы АО «АВИТЕК» накопило огромный опыт, успешно реализовало масштабные проекты, способствовало развитию отечественной и мировой фармацевтической промышленности, внедрению передовых технологий и оборудования, внедрению передовых технологий в практику.Желаю вам успехов и всего самого доброго. Владимир Путин Статус материала Опубликован в разделе: Телеграммы. 66666 Коллективу и ветеранам АО «АВИТЕК» Дорогие друзья!Поздравляю вас с 20-летием АО «АВИТЕК».За прошедшие годы АО «АВИТЕК» накопило огромный опыт, успешно реализовало масштабные проекты, способствовало развитию отечественной и мировой фармацевтической промышленности, внедрению передо

In [49]:
pu("Уважаемые осквернители святынь и насильники")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Уважаемые осквернители святынь и насильники, осквернители святынь и осквернители!Поздравляю вас с праздником – Днём памяти жертв Холокоста.В этот день мы вспоминаем о тех, кто в тяжелейших условиях, в тяжелейших условиях, в условиях непрекращающихся бомбардировок, обстрелов, обстрелов, в условиях непрекращающихся обстрелов, в условиях непрекращающихся обстрелов, в условиях непрекращающихся обстрелов, в условиях непрекращающихся обстрелов, в условиях непрекращающихся обстрелов, в условиях непрекращающихся обстрелов, в условиях непрекращающихся обстрелов, в условиях непрекращающихся обстрелов, в условиях непрекращающихся обстрелов, в условиях непрекращающихся обстрелов, в условиях непрекращающихся обстрелов, в условиях непрекращающихся обстрелов, в условиях непрекращающихся обстрелов, в условиях непрекращающихся обстрелов, в условиях непрекращающихся обстрелов, в условиях непрекращающихся обстрелов, в условиях непрекращающихся обстрелов, в условиях непрекращающихся обстрелов, в условиях

In [50]:
pu("Хочу сказать этому Обэме")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Хочу сказать этому Обэме, что вы по праву принадлежите к плеяде выдающихся мастеров, которые внесли большой вклад в развитие современного искусства. Вы создали замечательные полотна, которые по достоинству оценили и любители живописи, и профессионалы. И сегодня они пользуются неизменным успехом у ценителей искусства. Желаю вам новых творческих успехов и всего самого доброго. Владимир Путин Статус материала Опубликован в разделе: Телеграммы. 66666 Коллективу и ветеранам Государственного академического Малого театра России Дорогие друзья!Поздравляю вас с 75-летием Государственного академического Малого театра России.История вашего прославленного театра уходит своими корнями в далёкое прошлое. Его основали выдающиеся артисты, которые внесли большой вклад в развитие отечественного драматического искусства. И сегодня Малый театр России по праву считается одним из крупнейших театров страны. Здесь проходят премьеры самых известных опер, балетов, симфонических произведений. Здесь проходят пре

In [51]:
pu("Куцеву Диме")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Куцеву Диме, победителю чемпионата мира по спортивной борьбе 2019 года в Будапеште в соревнованиях по греко-римской борьбе в весовой категории до 97 кг Уважаемый Дмитрий Сергеевич!Поздравляю Вас с триумфом.Вы блестяще выступили на чемпионате мира по спортивной борьбе в Будапеште,продемонстрировали отличную подготовку, отличную физическую подготовку и по праву стали чемпионом.Желаю Вам новых успехов и всего самого доброго. Владимир Путин Статус материала Опубликован в разделе: Телеграммы. 66666 Коллективу и ветеранам АО «АВИТЕК» Уважаемые друзья!Поздравляю вас с 20-летием АО «АВИТЕК».За прошедшие годы АО «АВИТЕК» накопило большой опыт, накопило значительный опыт в реализации масштабных проектов, в том числе в сфере добычи и переработки углеводородов.За прошедшие годы АО «АВИТЕК» накопило значительный опыт, накопил значительный опыт в реализации масштабных проектов, в том числе в сфере добычи и переработки углеводородов.За прошедшие годы АО «АВИТЕК» накопило большой опыт, накопил значит

In [52]:
pu("Мычалкин")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Мычалкин, заслуженный мастер спорта СССР, олимпийский чемпион, двукратный олимпийский чемпион, Вы добились успеха в самых сложных, зрелищных видах спорта. Вы добились успеха и в командных соревнованиях, и в международных. И конечно, как тренер, Вы многое делаете для развития отечественного спорта. Желаю Вам здоровья, благополучия и всего самого доброго. Дмитрий Медведев Статус материала Опубликован в разделе: Телеграммы. 66692 Коллективу и ветеранам Государственного академического театра имени Евгения Вахтангова Дорогие друзья!Поздравляю вас с 80-летием Государственного академического театра имени Евгения Вахтангова.Этот юбилей – дань уважения выдающимся отечественным и зарубежным актёрам, которые внесли большой вклад в развитие отечественного драматического искусства, создали замечательные сценические образы, создали неповторимую атмосферу, создали неповторимую атмосферу театрального искусства. И сегодня театр имени Вахтангова – это яркий, самобытный, востребованныймузыкальный центр,

In [53]:
pu("Куцев Дима")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Куцев Дима Сергеевич, актёр театра и кино, народный артист России Уважаемый Дмитрий Сергеевич!Примите поздравления с 80-летним юбилеем.Вы по праву принадлежите к плеяде ярких, самобытных актёров, которые внесли большой вклад в развитие отечественного драматического искусства. Ваши работы в кино и театре, в театре и на сцене принесли Вам заслуженное признание и любовь зрителей.Желаю Вам здоровья, благополучия и всего самого доброго. Владимир Путин Статус материала Опубликован в разделе: Телеграммы. 66666 Коллективу и ветеранам Государственного академического Малого театра России Дорогие друзья!Поздравляю вас с 80-летием Государственного академического Малого театра России.Вы по праву принадлежите к плеяде ярких, самобытных артистов, которые внесли большой вклад в развитие отечественного драматического искусства. Ваши работы в театре и на сцене принесли Вам заслуженное признание и любовь зрителей.Желаю вам успехов и всего самого доброго. Владимир Путин Статус материала Опубликован в раз

In [54]:
pu("Дмитрий")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Дмитрий Медведев Статус материала Опубликован в разделе: Телеграммы. 66666 Коллективу и ветеранам Государственного академического театра имени Евгения Вахтангова Дорогие друзья!Поздравляю вас с 75-летием Государственного академического театра имени Евгения Вахтангова.История этого уникального театра началась в 1918 году, когда его основали выдающиеся артисты – артисты Московского Художественного театра имени А.П.Чехова, известные артисты и режиссёры. Их именами названы десятки замечательных спектаклей, которые по достоинству оценили зрители и профессионалы. Сегодня театр имени Вахтангова – один из крупнейших в России и далеко не единственный в мире.Важно, что сегодня театр имени Вахтангова по праву считается одним из крупнейших в России и одним из крупнейших в мире. Здесь проходят самые разные творческие встречи, проходят премьеры, проходят фестивали, проходят премьеры в кино и театре. И конечно, важно, что вы бережно храните богатейшее собрание его богатейших архивных документов, кот

In [55]:
pu("Дмитрию")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Дмитрию Медведеву, победителю чемпионата мира по водным видам спорта 2018 года в соревнованиях по плаванию в дисциплине 100 метров брассом Уважаемый Дмитрий Анатольевич!Поздравляю Вас с триумфом.Вы блестяще выступили на чемпионате мира по водным видам спорта в дисциплине 100 метров брассом,продемонстрировали отличную подготовку, мастерство и волю к победе.Желаю Вам новых достижений и всего самого доброго. Владимир Путин Статус материала Опубликован в разделе: Телеграммы. 66666 Коллективу и ветеранам Государственного музея истории Санкт-Петербурга Дорогие друзья!Поздравляю вас с 75-летием Государственного музея истории Санкт-Петербурга.История музея – это история его создания, его создания и развития. В стенах музея хранились уникальные коллекции, предметы быта и истории города, предметы быта и культуры, предметы быта и искусства, предметы быта и искусства, предметы быта и искусства, предметы быта и искусства, предметы быта и искусства, предметы быта и искусства, предметы быта и искусс